In [33]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from IPython.display import display, HTML

In [34]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [35]:
import numpy as np
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel

In [36]:
dataset = get_movielens_dataset(variant='100K')
print(dataset)

<Interactions dataset (944 users x 1683 items x 100000 interactions)>


In [37]:
import torch

In [64]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

#dataset = get_movielens_dataset(variant='100K')

#train, test = random_train_test_split(dataset)

#model = ImplicitFactorizationModel(n_iter=6, loss='bpr')


In [ ]:
print('Split into \n {} and \n {}.'.format(train, test))

In [ ]:
model.fit(train, verbose=True)

In [ ]:
mrr = mrr_score(model, test)

In [ ]:
from spotlight.evaluation import rmse_score

#train_mrr = mrr_score(model, train)
train_rmse = rmse_score(model, train)
#test_mrr = mrr_score(model, test)
test_rmse = rmse_score(model, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

In [ ]:
type(train_rmse)

In [ ]:
print(train_rmse)

In [38]:
raw_df = pd.read_csv('../data/export_with_basket_2017.csv', encoding = "utf-8")#, nrows = 100000)

In [39]:
## should we have any missing customer data
# fortunately this is not the case for us, but I will leave the cleaning step as must do one anyway
cleaned_df = raw_df.loc[pd.isnull(raw_df.CUS_N_KEY_CUSTOMER) == False]

In [40]:
item_lookup = cleaned_df[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']].drop_duplicates() # Only get unique item/description pairs
item_lookup['ART_C_PROD_NUMBER'] = item_lookup.ART_C_PROD_NUMBER.astype(int) # Encode as strings for future lookup ease
item_lookup['ART_V_ART_DESCRIPTION'] = item_lookup.ART_V_ART_DESCRIPTION.astype(str) # Encode as strings for future lookup ease

In [41]:
cleaned_df['CUS_N_KEY_CUSTOMER '] = cleaned_df.CUS_N_KEY_CUSTOMER.astype(int) # Convert to int for customer ID
cleaned_df = cleaned_df[['ART_C_PROD_NUMBER', 'SCM_D_QUANTITY', 'CUS_N_KEY_CUSTOMER']] # Get rid of unnecessary info
grouped_cleaned = cleaned_df.groupby(['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']).sum().reset_index() # Group together
grouped_cleaned.SCM_D_QUANTITY.loc[grouped_cleaned.SCM_D_QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('SCM_D_QUANTITY > 2') # Only get customers where purchase totals were positive

## If we look at our final resulting matrix of grouped purchases, we see the following:
grouped_purchased.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,CUS_N_KEY_CUSTOMER,ART_C_PROD_NUMBER,SCM_D_QUANTITY
0,22,52735,5.0
7,22,397041,4.0
8,22,399251,3.0
12,22,447409,4.0
18,22,543556,3.0


In [42]:
grouped_purchased.shape

(2094184, 3)

In [43]:
#grouped_purchased[['CUS_N_KEY_CUSTOMER_int', 'ART_C_PROD_NUMBER_int']] = grouped_purchased[['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']].astype('int32')
tuples = [(x[0].astype('int32'), x[1].astype('int32'),x[2]) for x in grouped_purchased.values]
u_ids = np.array([x[0].astype('int32') for x in grouped_purchased.values])
i_ids = np.array([x[1].astype('int32') for x in grouped_purchased.values])
w = np.array([x[2] for x in grouped_purchased.values])

In [44]:
u_ids

array([      22,       22,       22, ..., 18972790, 18972790, 18972790],
      dtype=int32)

In [45]:
i_ids

array([ 52735, 397041, 399251, ..., 577604, 580924, 607009], dtype=int32)

In [46]:
w

array([5., 4., 3., ..., 3., 3., 3.])

In [47]:
tuples[:5]

[(22, 52735, 5.0),
 (22, 397041, 4.0),
 (22, 399251, 3.0),
 (22, 447409, 4.0),
 (22, 543556, 3.0)]

In [48]:
def lower_limit(df, CUS_N_KEY_CUSTOMER_min, ART_C_PROD_NUMBER_min):
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of items: {}'.format(n_items))
    print('Weak-Sparsity: {:4.3f}%'.format(sparsity)) # this is not exactly a sparsity. I have to change it a math correct one
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        ART_C_PROD_NUMBER_counts = df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count()
        df = df[~df.CUS_N_KEY_CUSTOMER.isin(ART_C_PROD_NUMBER_counts[ART_C_PROD_NUMBER_counts < ART_C_PROD_NUMBER_min].index.tolist())]
        CUS_N_KEY_CUSTOMER_counts = df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count()
        df = df[~df.ART_C_PROD_NUMBER.isin(CUS_N_KEY_CUSTOMER_counts[CUS_N_KEY_CUSTOMER_counts < CUS_N_KEY_CUSTOMER_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count().min() >= ART_C_PROD_NUMBER_min)
    assert(df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count().min() >= CUS_N_KEY_CUSTOMER_min)
    
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of items: {}'.format(n_items))
    print('Weak-Sparsity: {:4.3f}%'.format(sparsity)) # this is not exactly a sparsity. I have to change it a math correct one
    return df

In [49]:
df_lim = lower_limit(grouped_purchased, 5, 5)

Starting likes info
Number of users: 594238
Number of items: 6869
Weak-Sparsity: 0.051%
Ending likes info
Number of users: 137953
Number of items: 5564
Weak-Sparsity: 0.163%


In [50]:
customers = list(np.sort(df_lim.CUS_N_KEY_CUSTOMER.unique())) # Get our unique customers
products = list(df_lim.ART_C_PROD_NUMBER.unique()) # Get our unique products that were purchased
quantity = list(df_lim.SCM_D_QUANTITY) # All of our purchases

rows = df_lim.CUS_N_KEY_CUSTOMER.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = df_lim.ART_C_PROD_NUMBER.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [51]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.83719241255042

In [52]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

In [53]:
purchases_sparse.shape

(137953, 5564)

In [67]:
model1 = ImplicitFactorizationModel(n_iter=6, loss='bpr')

In [68]:
from spotlight.interactions import Interactions

In [69]:
grouped_purchased_interactions = Interactions(user_ids=u_ids,
                                              item_ids=i_ids,
                                              weights = w)

In [70]:
print(grouped_purchased_interactions)

<Interactions dataset (18972791 users x 660009 items x 2094184 interactions)>


In [71]:
np.array(tuples[0])

array([2.2000e+01, 5.2735e+04, 5.0000e+00])

In [72]:
np.max(u_ids)

18972790

In [73]:
type(w[0])

numpy.float64

In [74]:
model1.fit(grouped_purchased_interactions, verbose = True)

KeyboardInterrupt: 

In [66]:
print(model1._net)

cnn
